# LangGraph Advanced – Part 3️⃣: Fine‑Tuning via Ollama

Claude sinh dữ liệu, Ollama fine‑tune model cục bộ bằng LoRA, rồi đánh giá.

In [ ]:
%pip install -q langgraph langchain langchain-anthropic tiktoken

In [ ]:
import subprocess, json, uuid
from pathlib import Path
from langgraph.graph import StateGraph, MessageState
from langchain_anthropic import ChatAnthropic
from langchain.schema import HumanMessage

WORK_DIR = Path("ft_work"); WORK_DIR.mkdir(exist_ok=True)
DATA_FILE = WORK_DIR/"train.jsonl"
MODEL_BASE = "llama2:7b"
MODEL_FT = "llama2_ft"

llm_c = ChatAnthropic(model_name="claude-3-opus-20240229", temperature=0.3)

class FTState(MessageState):
    step: str | None = None

def gen_data(state: FTState):
    pairs = []
    for i in range(20):
        q = f"Hỏi {i}: Ngân hàng là gì?"
        a = llm_c([HumanMessage(content=f"Trả lời ngắn gọn: {q}")]).content
        pairs.append({"prompt": q, "completion": a})
    DATA_FILE.write_text("\n".join(json.dumps(p) for p in pairs))
    return state

def tune(state: FTState):
    cmd = ["ollama", "create", MODEL_FT, "--from", MODEL_BASE,
           "--lora", str(DATA_FILE)]
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)
    return state

def eval(state: FTState):
    test_q = "Ngân hàng số là gì?"
    out = subprocess.check_output(["ollama", "run", MODEL_FT, test_q])
    print("Model says:", out.decode()[:200])
    return state

g = StateGraph(FTState)
g.add_node("data", gen_data)
g.add_node("tune", tune)
g.add_node("eval", eval)
g.set_entry_point("data")
g.add_edge("data", "tune")
g.add_edge("tune", "eval")
g.compile().invoke(FTState())
